# Étape 3 — Indicateurs Clés de Performance (ICP)

L’objectif de cette étape est de calculer et analyser les principaux indicateurs
de performance commerciale et de satisfaction client à partir des données
nettoyées issues de l’étape précédente.

Les indicateurs sont construits en privilégiant des mesures économiquement
pertinentes, notamment des taux calculés en valeur lorsque cela est justifié.


**Import des libraries**

In [ ]:
# Import des librairies nécessaires à l'analyse des ICP

import pandas as pd
import numpy as np

# Options d'affichage pour une meilleure lisibilité
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", "{:,.2f}".format)


**Chargement des données nettoyées**

In [ ]:
# Chargement des données nettoyées depuis le dossier data/processed

order_lines = pd.read_csv("../data/processed/order_lines_clean.csv")
customers = pd.read_csv("../data/processed/customers_clean.csv")
products = pd.read_csv("../data/processed/products_clean.csv")

**Préparation des données pour l'analyse des ICP** 

In [ ]:
# Conversion des variables de date
order_lines["order_date"] = pd.to_datetime(order_lines["order_date"])

# Création de la variable mois de commande
order_lines["order_month"] = order_lines["order_date"].dt.to_period("M")

# Vérification rapide de la structure des données
order_lines[[
    "order_id",
    "order_date",
    "order_month",
    "net_amount",
    "quantity",
    "returned",
    "category",
    "channel",
    "review_score",
    "delivery_days"
]].head()


**Chiffre d'affaires total (global)**

In [ ]:
# Calcul du chiffre d'affaires total (CA)
# Le CA est calculé à partir du montant net (net_amount),
# c’est-à-dire après application des remises.

ca_total = order_lines["net_amount"].sum()

# Affichage du résultat
print(f"Chiffre d'affaires total (CA net) : {ca_total:,.2f}")

**Chiffre d'affaires total (mensuel)**

In [ ]:
# Calcul du chiffre d'affaires mensuel à partir du CA net
# Agrégation par mois de commande (order_month)

ca_mensuel = (
    order_lines
    .groupby("order_month", as_index=False)
    .agg(CA_mensuel=("net_amount", "sum"))
    .sort_values("order_month")
)

# Affichage du chiffre d'affaires mensuel
ca_mensuel

**Panier moyen global (AOV)**

In [ ]:
# Calcul du panier moyen global (Average Order Value - AOV)
# AOV = Chiffre d'affaires total / Nombre de commandes distinctes

# Nombre total de commandes
nb_commandes = order_lines["order_id"].nunique()

# Panier moyen global
aov_global = ca_total / nb_commandes

# Affichage du résultat
print(f"Panier moyen global (AOV) : {aov_global:,.2f}")

**Panier moyen mensuel (AOV mensuel)**

In [ ]:
# Calcul du panier moyen mensuel (AOV mensuel)
# AOV mensuel = CA mensuel / Nombre de commandes mensuelles

aov_mensuel = (
    order_lines
    .groupby("order_month")
    .agg(
        CA_mensuel=("net_amount", "sum"),
        nb_commandes=("order_id", "nunique")
    )
    .assign(AOV_mensuel=lambda df: df["CA_mensuel"] / df["nb_commandes"])
    .reset_index()
    .sort_values("order_month")
)

# Affichage du panier moyen mensuel
aov_mensuel

## Méthodologie de calcul du taux de remise

Les taux de remise peuvent être calculés soit en volume (nombre d’articles),
soit en valeur monétaire. Dans le cadre de ce projet, le calcul est effectué
**en valeur**, afin de tenir compte de l’hétérogénéité des prix entre les
produits et les catégories.

Le calcul en volume attribuerait le même poids à des articles de faible valeur
et à des articles de valeur élevée, ce qui pourrait biaiser l’analyse
économique. À l’inverse, le calcul en valeur permet de mesurer l’impact réel
des remises sur le chiffre d’affaires.

Les définitions retenues sont les suivantes :

- **Prix catalogue** :  
  \[
  \text{Prix catalogue} = \text{unit\_price} \times \text{quantity}
  \]

- **Prix vendu** :  
  \[
  \text{Prix vendu} = \text{net\_amount}
  \]

Le taux de remise est alors défini comme :

\[
\text{Taux de remise} =
\frac{\sum (\text{Prix catalogue} - \text{Prix vendu})}
{\sum \text{Prix catalogue}}
\]

Cette approche garantit une interprétation économiquement pertinente et
cohérente avec les objectifs d’analyse du projet.

**Taux de remise moyen global**

In [ ]:
# Calcul du taux de remise moyen global (en valeur)

# Calcul du prix catalogue (avant remise)
order_lines["catalog_amount"] = order_lines["unit_price"] * order_lines["quantity"]

# Calcul du montant total des remises
total_discount_value = (order_lines["catalog_amount"] - order_lines["net_amount"]).sum()

# Calcul du montant total catalogue
total_catalog_value = order_lines["catalog_amount"].sum()

# Taux de remise moyen global (en valeur)
taux_remise_global = total_discount_value / total_catalog_value

# Affichage du résultat
print(f"Taux de remise moyen global (en valeur) : {taux_remise_global:.2%}")